### Hierarchical econder for intent classification

#### Downloading dataset

In [1]:
from datasets import load_dataset
from transformers import BertModel, BertTokenizer

import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
data = load_dataset("swda")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/213543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/56729 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4514 [00:00<?, ? examples/s]

Dataset swda downloaded and prepared to /root/.cache/huggingface/datasets/swda/default/0.0.0/b53d17ec4c6e31d0921591dd2d8e86d15850822209a980fcddb2983fc948e499. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['swda_filename', 'ptb_basename', 'conversation_no', 'transcript_index', 'act_tag', 'damsl_act_tag', 'caller', 'utterance_index', 'subutterance_index', 'text', 'pos', 'trees', 'ptb_treenumbers', 'talk_day', 'length', 'topic_description', 'prompt', 'from_caller', 'from_caller_sex', 'from_caller_education', 'from_caller_birth_year', 'from_caller_dialect_area', 'to_caller', 'to_caller_sex', 'to_caller_education', 'to_caller_birth_year', 'to_caller_dialect_area'],
        num_rows: 213543
    })
    validation: Dataset({
        features: ['swda_filename', 'ptb_basename', 'conversation_no', 'transcript_index', 'act_tag', 'damsl_act_tag', 'caller', 'utterance_index', 'subutterance_index', 'text', 'pos', 'trees', 'ptb_treenumbers', 'talk_day', 'length', 'topic_description', 'prompt', 'from_caller', 'from_caller_sex', 'from_caller_education', 'from_caller_birth_year', 'from_caller_dialect_area', 'to_caller', 'to_caller_sex', 'to_caller_educ

In [5]:
train_data = data['train']
train_df = pd.DataFrame(train_data)

In [6]:
train_df

,swda_filename,ptb_basename,conversation_no,transcript_index,act_tag,damsl_act_tag,caller,utterance_index,subutterance_index,text,...,from_caller,from_caller_sex,from_caller_education,from_caller_birth_year,from_caller_dialect_area,to_caller,to_caller_sex,to_caller_education,to_caller_birth_year,to_caller_dialect_area
0,sw00utt/sw_0001_4325.utt,4/sw4325,4325,0,115,26,A,1,1,Okay. /,...,1632,FEMALE,2,1962,WESTERN,1519,FEMALE,1,1971,SOUTH MIDLAND
1,sw00utt/sw_0001_4325.utt,4/sw4325,4325,1,82,15,A,1,2,"{D So, }",...,1632,FEMALE,2,1962,WESTERN,1519,FEMALE,1,1971,SOUTH MIDLAND
2,sw00utt/sw_0001_4325.utt,4/sw4325,4325,2,206,36,B,2,1,"[ [ I guess, +",...,1632,FEMALE,2,1962,WESTERN,1519,FEMALE,1,1971,SOUTH MIDLAND
3,sw00utt/sw_0001_4325.utt,4/sw4325,4325,3,148,20,A,3,1,"What kind of experience [ do you, + do you ] h...",...,1632,FEMALE,2,1962,WESTERN,1519,FEMALE,1,1971,SOUTH MIDLAND
4,sw00utt/sw_0001_4325.utt,4/sw4325,4325,4,148,20,B,4,1,"I think, ] + {F uh, } I wonder ] if that worke...",...,1632,FEMALE,2,1962,WESTERN,1519,FEMALE,1,1971,SOUTH MIDLAND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213538,sw11utt/sw_1115_3451.utt,3/sw3451,3451,161,125,4,B,94,1,It really is. /,...,1415,FEMALE,1,1939,SOUTH MIDLAND,1423,FEMALE,2,1960,WESTERN
213539,sw11utt/sw_1115_3451.utt,3/sw3451,3451,162,148,20,A,95,1,And have enthusiasm. /,...,1415,FEMALE,1,1939,SOUTH MIDLAND,1423,FEMALE,2,1960,WESTERN
213540,sw11utt/sw_1115_3451.utt,3/sw3451,3451,163,194,21,B,96,1,"Yeah, /",...,1415,FEMALE,1,1939,SOUTH MIDLAND,1423,FEMALE,2,1960,WESTERN
213541,sw11utt/sw_1115_3451.utt,3/sw3451,3451,164,176,4,B,96,2,it really is. /,...,1415,FEMALE,1,1939,SOUTH MIDLAND,1423,FEMALE,2,1960,WESTERN


#### Tokenization

In [7]:
# Load the pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', padding=True, truncation=True)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [8]:
# Define the data preprocessing function
def preprocess_data(utterance):
    text = utterance['text']
    label = utterance['act_tag']
    return {'text': text, 'label': label}

In [9]:
# Preprocess the data
preprocessed_dataset = data.map(preprocess_data)

  0%|          | 0/213543 [00:00<?, ?ex/s]

  0%|          | 0/56729 [00:00<?, ?ex/s]

  0%|          | 0/4514 [00:00<?, ?ex/s]

In [10]:
preprocessed_dataset = preprocessed_dataset.remove_columns(['swda_filename', 'ptb_basename', 'conversation_no', 'transcript_index', 'act_tag', 'damsl_act_tag', 'caller', 'utterance_index', 'subutterance_index', 'pos', 'trees', 'ptb_treenumbers', 'talk_day', 'length', 'topic_description', 'prompt', 'from_caller', 'from_caller_sex', 'from_caller_education', 'from_caller_birth_year', 'from_caller_dialect_area', 'to_caller', 'to_caller_sex', 'to_caller_education', 'to_caller_birth_year', 'to_caller_dialect_area'])

In [11]:
preprocessed_dataset['train']['text'][:10]

['Okay.  /',
 '{D So, }',
 '[ [ I guess, +',
 'What kind of experience [ do you, + do you ] have, then with child care? /',
 'I think, ] + {F uh, } I wonder ] if that worked. /',
 'Does it say something? /',
 'I think it usually does.  /',
 'You might try, {F uh, }  /',
 "I don't know,  /",
 'hold it down a little longer,  /']

In [12]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, return_tensors='pt')

In [13]:
tokenized_dataset = preprocessed_dataset.map(lambda examples: tokenizer(examples['text'], truncation=True, padding=True), batched=True)

  0%|          | 0/214 [00:00<?, ?ba/s]

  0%|          | 0/57 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [14]:
tokenized_dataset['train'][0]

{'text': 'Okay.  /',
 'label': 115,
 'input_ids': [101,
  3100,
  1012,
  1013,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


#### Defining the hierarchical encoder architecture

The architecture of the hierarchical encoder consists of a **BERT-based model** with a linear classification layer on top, which has been fine-tuned on the SWDA dataset for intent classification.

By using the outputs of the different layers of the BERT model, the hierarchical encoder is able to capture the hierarchical structure of natural language, and the fine-tuning process allows the model to learn task-specific representations that are optimized for intent classification.

In [15]:
class HierarchicalEncoder(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.mlp = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size),
            nn.ReLU(),
            nn.Linear(self.bert.config.hidden_size, num_classes)
        )
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids.to(device), attention_mask=attention_mask.to(device))
        sequence_output = outputs.last_hidden_state
        pooled_output = torch.mean(sequence_output, dim=1)
        logits = self.mlp(pooled_output)
        return logits.to(device)



In [20]:
# Define the data collator
from transformers import DataCollatorWithPadding
lst = tokenized_dataset['train']['input_ids']


from torch.nn.utils.rnn import pad_sequence

# Define the data collator
def collate_fn(examples):
    input_ids = pad_sequence([torch.tensor(x['input_ids']) for x in examples], batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence([torch.tensor(x['attention_mask']) for x in examples], batch_first=True, padding_value=0)
    labels = torch.tensor([x['label'] for x in examples])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': labels}

# Define the training and validation data loaders
train_loader = DataLoader(tokenized_dataset['train'], batch_size=64, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(tokenized_dataset['validation'], batch_size=64, collate_fn=collate_fn)


In [17]:
# Define the batch_sizemodel, loss function, and optimizer
model = HierarchicalEncoder(num_classes=len(data['train'].features['act_tag'].names)).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
# Train the model
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    count = 0
    for i, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * input_ids.size(0)
        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Train Loss: {loss.item():.4f}')

    train_loss /= len(tokenized_dataset['train'])
    
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            val_loss += loss.item() * input_ids.size(0)
            preds = torch.argmax(logits, axis=1)
            correct += torch.sum(preds == labels)
            total += input_ids.size(0)
    val_loss /= len(tokenized_dataset['validation'])
    val_acc = correct / total
    
    
    print(f'Epoch {epoch + 1}, train loss: {train_loss:.4f}, val loss: {val_loss:.4f}, val accuracy: {val_acc:.4f}')

RuntimeError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 15.90 GiB total capacity; 14.70 GiB already allocated; 31.75 MiB free; 15.08 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF